## An illustration of Ray Tune to optimize the parameters of a simple Neural Net.
We use the https://janakiev.com/blog/pytorch-iris/ implementation of a multilayer neural net to classify the Iris dataset.   


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
plt.style.use('ggplot')
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

We first load the data.

In [2]:
iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

# Scale data to have mean 0 and variance 1 
# which is importance for convergence of the neural network
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data set into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)
X_train = Variable(torch.from_numpy(X_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()
X_test  = Variable(torch.from_numpy(X_test)).float()
y_test  = Variable(torch.from_numpy(y_test)).long()



we need to create (data, target) pairs for input to our tune program.   In the abive X_train and X_test are the iris data and y_train and y_test are the targets.

In [5]:
def load_data(data_dir=None):
    traindata = []
    for i in range(X_train.shape[0]):
        traindata.append((X_train[i], y_train[i]))
    testdata = []
    for i in range(X_test.shape[0]):
        testdata.append((X_test[i], y_test[i]))
    return traindata, testdata

The Model is very simple.   it has three linear layers where the input is of dimension four and the output has dimension 3.   two parameters l1 and l2 specify the other dimensions.

In [7]:
class Model(nn.Module):
    def __init__(self, l1=4, l2=4):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(4, l1)
        self.layer2 = nn.Linear(l1, l2)
        self.layer3 = nn.Linear(l2, 3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

In [8]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from torch.utils.data import random_split

In [20]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

For Tune we need a configuration file to define the parameter space we want to explore.

We also need to wrap the model in a function that Tune will call.  The wrap_model function does a standard SGD/Adam learning and validing pass for the model.  

In [10]:
config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 6)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 6)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }

def wrap_model(config, data_dir=None):
    # Hyperparameters
    net = Model(config["l1"], config["l2"])
    optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"])
    criterion   = nn.CrossEntropyLoss()
       
    trainset, testset = load_data(data_dir)
    test_abs = int(len(trainset) * 0.8)
    
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    
    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)

    for epoch in range(50):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
        #print("loss=",(val_loss / val_steps), " accuracy=",correct / total)

    print("Finished Training")


In [11]:
from functools import partial

The core of the tune process is the call tune.run() which need a scheduler,  a reporter and the wrap_model function.  note: looking at the results you will find several configurations give a validation accuracy of 1.0.   the one selected has the minimun loss.   this is a trivial problems, so an accuracy of 1.0 is not special.  the data set is tiny.

In [12]:
device = 'cpu'
scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=10,
        grace_period=1,
        reduction_factor=2)

reporter = CLIReporter(
        metric_columns=["loss", "accuracy", "training_iteration"])

result = tune.run(
        partial(wrap_model, data_dir='./data'),
        resources_per_trial={"cpu": 2, "gpu": 0},
        config=config,
        num_samples=50,
        scheduler=scheduler,
        progress_reporter=reporter)

print(result)

2021-04-07 10:49:43,370	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265
2021-04-07 10:49:53,926	WARNING experiment.py:284 -- No name detected on trainable. Using DEFAULT.
2021-04-07 10:49:53,926	INFO registry.py:64 -- Detected unknown callable for trainable. Converting to class.
2021-04-07 10:49:53,926	WARNING function_runner.py:539 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-04-07 10:49:54,332	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


== Status ==
Memory usage on this node: 9.3/15.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2/8 CPUs, 0/0 GPUs, 0.0/4.1 GiB heap, 0.0/1.42 GiB objects
Result logdir: C:\Users\ganno\ray_results\DEFAULT_2021-04-07_10-49-53
Number of trials: 1/50 (1 RUNNING)
+---------------------+----------+-------+--------------+------+------+------------+
| Trial name          | status   | loc   |   batch_size |   l1 |   l2 |         lr |
|---------------------+----------+-------+--------------+------+------+------------|
| DEFAULT_a8b27_00000 | RUNNING  |       |            2 |    8 |   16 | 0.00151791 |
+---------------------+----------+-------+--------------+------+------+------------+


Result for DEFAULT_a8b27_00001:
  accuracy: 0.20833333333333334
  date: 2021-04-07_10-50-09
  done: false
  experiment_id: d19450caf2fb4666ac8d5dd770b0e658
  hostname: DESKTOP-LLL5ELU
  iterations_since_restore: 1
  

Result for DEFAULT_a8b27_00001:
  accuracy: 0.375
  date: 2021-04-07_10-50-41
  done: false
  experiment_id: d19450caf2fb4666ac8d5dd770b0e658
  hostname: DESKTOP-LLL5ELU
  iterations_since_restore: 3
  loss: 1.0083849728107452
  node_ip: 10.0.0.30
  pid: 16524
  should_checkpoint: true
  time_since_restore: 45.75669002532959
  time_this_iter_s: 16.5803701877594
  time_total_s: 45.75669002532959
  timestamp: 1617817841
  timesteps_since_restore: 0
  training_iteration: 3
  trial_id: a8b27_00001
  
== Status ==
Memory usage on this node: 9.8/15.6 GiB
Using AsyncHyperBand: num_stopped=4
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: -0.8599099268515904 | Iter 1.000: -1.0795202851295471
Resources requested: 8/8 CPUs, 0/0 GPUs, 0.0/4.1 GiB heap, 0.0/1.42 GiB objects
Result logdir: C:\Users\ganno\ray_results\DEFAULT_2021-04-07_10-49-53
Number of trials: 9/50 (1 PENDING, 4 RUNNING, 4 TERMINATED)
+---------------------+------------+-----------------+--------------+------+------+---

Result for DEFAULT_a8b27_00007:
  accuracy: 0.6666666666666666
  date: 2021-04-07_10-51-10
  done: false
  experiment_id: d14cd56cfc6a40ae9ad3f542b2a9782c
  hostname: DESKTOP-LLL5ELU
  iterations_since_restore: 2
  loss: 0.837928831577301
  node_ip: 10.0.0.30
  pid: 18516
  should_checkpoint: true
  time_since_restore: 41.31043243408203
  time_this_iter_s: 22.996931552886963
  time_total_s: 41.31043243408203
  timestamp: 1617817870
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: a8b27_00007
  
== Status ==
Memory usage on this node: 9.7/15.6 GiB
Using AsyncHyperBand: num_stopped=5
Bracket: Iter 8.000: None | Iter 4.000: -0.8168812245130539 | Iter 2.000: -0.837928831577301 | Iter 1.000: -1.0795202851295471
Resources requested: 8/8 CPUs, 0/0 GPUs, 0.0/4.1 GiB heap, 0.0/1.42 GiB objects
Result logdir: C:\Users\ganno\ray_results\DEFAULT_2021-04-07_10-49-53
Number of trials: 10/50 (1 PENDING, 4 RUNNING, 5 TERMINATED)
+---------------------+------------+-----------------+---

Result for DEFAULT_a8b27_00007:
  accuracy: 0.75
  date: 2021-04-07_10-51-36
  done: false
  experiment_id: d14cd56cfc6a40ae9ad3f542b2a9782c
  hostname: DESKTOP-LLL5ELU
  iterations_since_restore: 3
  loss: 0.788239598274231
  node_ip: 10.0.0.30
  pid: 18516
  should_checkpoint: true
  time_since_restore: 67.12062048912048
  time_this_iter_s: 25.810188055038452
  time_total_s: 67.12062048912048
  timestamp: 1617817896
  timesteps_since_restore: 0
  training_iteration: 3
  trial_id: a8b27_00007
  
== Status ==
Memory usage on this node: 9.7/15.6 GiB
Using AsyncHyperBand: num_stopped=5
Bracket: Iter 8.000: None | Iter 4.000: -0.8168812245130539 | Iter 2.000: -0.837928831577301 | Iter 1.000: -1.0725146134694417
Resources requested: 8/8 CPUs, 0/0 GPUs, 0.0/4.1 GiB heap, 0.0/1.42 GiB objects
Result logdir: C:\Users\ganno\ray_results\DEFAULT_2021-04-07_10-49-53
Number of trials: 10/50 (1 PENDING, 4 RUNNING, 5 TERMINATED)
+---------------------+------------+-----------------+--------------+--

...

Result for DEFAULT_a8b27_00049:
  accuracy: 0.7916666666666666
  date: 2021-04-07_11-15-04
  done: false
  experiment_id: 6d577f1b122d42b185fd352494687263
  hostname: DESKTOP-LLL5ELU
  iterations_since_restore: 2
  loss: 0.6993801295757294
  node_ip: 10.0.0.30
  pid: 936
  should_checkpoint: true
  time_since_restore: 44.668556451797485
  time_this_iter_s: 19.27468752861023
  time_total_s: 44.668556451797485
  timestamp: 1617819304
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: a8b27_00049
  
== Status ==
Memory usage on this node: 9.1/15.6 GiB
Using AsyncHyperBand: num_stopped=48
Bracket: Iter 8.000: -0.5956321209669113 | Iter 4.000: -0.6098987658818563 | Iter 2.000: -0.7845200747251511 | Iter 1.000: -1.0526352177063625
Resources requested: 4/8 CPUs, 0/0 GPUs, 0.0/4.1 GiB heap, 0.0/1.42 GiB objects
Result logdir: C:\Users\ganno\ray_results\DEFAULT_2021-04-07_10-49-53
Number of trials: 50/50 (2 RUNNING, 48 TERMINATED)
+---------------------+------------+--------------

Result for DEFAULT_a8b27_00049:
  accuracy: 0.9583333333333334
  date: 2021-04-07_11-15-22
  done: false
  experiment_id: 6d577f1b122d42b185fd352494687263
  hostname: DESKTOP-LLL5ELU
  iterations_since_restore: 3
  loss: 0.6078685224056244
  node_ip: 10.0.0.30
  pid: 936
  should_checkpoint: true
  time_since_restore: 62.5729238986969
  time_this_iter_s: 17.904367446899414
  time_total_s: 62.5729238986969
  timestamp: 1617819322
  timesteps_since_restore: 0
  training_iteration: 3
  trial_id: a8b27_00049
  
== Status ==
Memory usage on this node: 9.1/15.6 GiB
Using AsyncHyperBand: num_stopped=48
Bracket: Iter 8.000: -0.5956321209669113 | Iter 4.000: -0.6098987658818563 | Iter 2.000: -0.7845200747251511 | Iter 1.000: -1.0526352177063625
Resources requested: 4/8 CPUs, 0/0 GPUs, 0.0/4.1 GiB heap, 0.0/1.42 GiB objects
Result logdir: C:\Users\ganno\ray_results\DEFAULT_2021-04-07_10-49-53
Number of trials: 50/50 (2 RUNNING, 48 TERMINATED)
+---------------------+------------+-----------------

Result for DEFAULT_a8b27_00048:
  accuracy: 1.0
  date: 2021-04-07_11-15-44
  done: false
  experiment_id: 0fb4696268e545858bec78e5d39e87fe
  hostname: DESKTOP-LLL5ELU
  iterations_since_restore: 7
  loss: 0.556784967581431
  node_ip: 10.0.0.30
  pid: 16924
  should_checkpoint: true
  time_since_restore: 177.12867975234985
  time_this_iter_s: 17.04256010055542
  time_total_s: 177.12867975234985
  timestamp: 1617819344
  timesteps_since_restore: 0
  training_iteration: 7
  trial_id: a8b27_00048
  
Result for DEFAULT_a8b27_00048:
  accuracy: 1.0
  date: 2021-04-07_11-15-57
  done: false
  experiment_id: 0fb4696268e545858bec78e5d39e87fe
  hostname: DESKTOP-LLL5ELU
  iterations_since_restore: 8
  loss: 0.5538281053304672
  node_ip: 10.0.0.30
  pid: 16924
  should_checkpoint: true
  time_since_restore: 189.60955452919006
  time_this_iter_s: 12.48087477684021
  time_total_s: 189.60955452919006
  timestamp: 1617819357
  timesteps_since_restore: 0
  training_iteration: 8
  trial_id: a8b27_0004

2021-04-07 11:16:24,182	INFO tune.py:448 -- Total run time: 1601.29 seconds (1589.82 seconds for the tuning loop).


Result for DEFAULT_a8b27_00048:
  accuracy: 1.0
  date: 2021-04-07_11-16-24
  done: true
  experiment_id: 0fb4696268e545858bec78e5d39e87fe
  hostname: DESKTOP-LLL5ELU
  iterations_since_restore: 10
  loss: 0.5625380625327429
  node_ip: 10.0.0.30
  pid: 16924
  should_checkpoint: true
  time_since_restore: 216.61927723884583
  time_this_iter_s: 14.58387279510498
  time_total_s: 216.61927723884583
  timestamp: 1617819384
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: a8b27_00048
  
== Status ==
Memory usage on this node: 8.7/15.6 GiB
Using AsyncHyperBand: num_stopped=50
Bracket: Iter 8.000: -0.5952872435251871 | Iter 4.000: -0.6120609839757284 | Iter 2.000: -0.7845200747251511 | Iter 1.000: -1.0526352177063625
Resources requested: 2/8 CPUs, 0/0 GPUs, 0.0/4.1 GiB heap, 0.0/1.42 GiB objects
Result logdir: C:\Users\ganno\ray_results\DEFAULT_2021-04-07_10-49-53
Number of trials: 50/50 (1 RUNNING, 49 TERMINATED)
+---------------------+------------+-----------------+--------

In [13]:
import os
best_trial = result.get_best_trial("loss", "min", "last")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
    best_trial.last_result["loss"]))
print("Best trial final validation accuracy: {}".format(
    best_trial.last_result["accuracy"]))

best_trained_model = Model(best_trial.config["l1"], best_trial.config["l2"])
device = "cpu"

best_checkpoint_dir = best_trial.checkpoint.value
model_state, optimizer_state = torch.load(os.path.join(
    best_checkpoint_dir, "checkpoint"))
best_trained_model.load_state_dict(model_state)

test_acc = test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))


Best trial config: {'l1': 4, 'l2': 32, 'lr': 0.05880677763477918, 'batch_size': 16}
Best trial final validation loss: 0.5520099997520447
Best trial final validation accuracy: 1.0
Best trial test set accuracy: 0.9333333333333333
